In [1]:
import time
from scrapy.crawler import CrawlerProcess
import json          
import logging
import scrapy
from scrapy.spiders import CrawlSpider, Rule
import urllib
import redis 
class crawlImages(scrapy.Spider):
    def parse(self, response): # When writing crawl spider rules, avoid using parse as callback, since the CrawlSpider uses the parse method itself to implement its logic. So if you override the parse method, the crawl spider will no longer work.
        for request_or_item in CrawlSpider.parse(self, response):
            if isinstance(request_or_item, scrapy.Request):
                request_or_item = request_or_item.replace(meta = {'start_url': response.meta['start_url']})
            yield request_or_item
    def make_requests_from_url(self, url):
        """A method that receives a URL and returns a Request object (or a list of Request objects) to scrape. 
        This method is used to construct the initial requests in the start_requests() method, 
        and is typically used to convert urls to requests.
        """
        return scrapy.Request(url, dont_filter=True, meta = {'start_url': url})
    def parse(self, response):
        raw_image_urls = response.xpath(".//img/@src").getall()
        clean_image_urls = []
        for img_url in raw_image_urls:
            clean_image_urls.append(response.urljoin(img_url))
        yield {'image_urls': clean_image_urls, 
              'start_url': response.meta['start_url']}

class RedisPipeline(object):  
    @classmethod 
    def from_crawler(cls, crawler): 
        return cls() 

    def open_spider(self, spider): 
        self.client = redis.Redis(host='localhost', port=6379, db=0)

    def close_spider(self, spider): 
            pass

    def process_item(self, item, spider): 
        start_url = item['start_url']
        self.client.lpush(start_url, *item['image_urls'])
        return item

In [2]:
ITEM_PIPELINES = {
   RedisPipeline: 600,
}
process = CrawlerProcess({
    'USER_AGENT': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36", 
    "ITEM_PIPELINES": ITEM_PIPELINES})

process.crawl(crawlImages, name = 'second_spider', start_urls = ['https://www.nytimes.com/ca/'])
process.start()
process.join()

2021-09-20 23:57:00 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-09-20 23:57:00 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.10 (default, Jun  2 2021, 10:49:15) - [GCC 9.4.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Linux-4.4.0-19041-Microsoft-x86_64-with-glibc2.29
2021-09-20 23:57:00 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-09-20 23:57:00 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Chrome/34.0.1847.131 Safari/537.36'}
2021-09-20 23:57:00 [scrapy.extensions.telnet] INFO: Telnet Password: a3cd702703315ed4
2021-09-20 23:57:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.M

2021-09-20 23:57:01 [scrapy.core.engine] INFO: Closing spider (finished)
2021-09-20 23:57:01 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 293,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 163974,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 0.300523,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 9, 21, 3, 57, 1, 37830),
 'httpcompression/response_bytes': 910607,
 'httpcompression/response_count': 1,
 'item_scraped_count': 1,
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'log_count/WARNING': 1,
 'memusage/max': 80252928,
 'memusage/startup': 80252928,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2021, 9, 21, 3, 57, 0, 737307)}
2021-09-20 23:57:01 [scrapy.core.engine] INFO: Spi

<Deferred at 0x7f21d20a8c40 current result: None>

In [3]:
tmp = redis.Redis(host='localhost', port=6379, db=0)

In [4]:
tmp.lrange('https://www.nytimes.com/ca/', 0, -1)

[b'https://static01.nyt.com/images/2020/03/24/crosswords/vertex-nytgames-hi-res/vertex-nytgames-hi-res-square320-v5.png?format=pjpg&quality=75&auto=webp&disable=upscale',
 b'https://static01.nyt.com/images/2020/03/23/crosswords/tiles-logo-nytgames-hi-res/tiles-logo-nytgames-hi-res-square320-v4.png?format=pjpg&quality=75&auto=webp&disable=upscale',
 b'https://static01.nyt.com/images/2021/05/27/multimedia/alpha-letterboxed-promo-1622145789727/alpha-letterboxed-promo-1622145789727-square320.png?format=pjpg&quality=75&auto=webp&disable=upscale',
 b'https://static01.nyt.com/images/2021/08/03/crosswords/nyt-games-homepage-playmodule-subscribe/nyt-games-homepage-playmodule-subscribe-square320.png?format=pjpg&quality=75&auto=webp&disable=upscale',
 b'https://static01.nyt.com/images/2020/03/23/crosswords/crossword-logo-nytgames-hires/crossword-logo-nytgames-hires-square320-v3.png?format=pjpg&quality=75&auto=webp&disable=upscale',
 b'https://static01.nyt.com/images/2019/04/18/homepage/spelling-b